# Supply Chain Optimization using a Transshipment Model

## 1. Project Goal

The primary goal of this project is to determine the most cost-effective shipping strategy for a multi-echelon distribution network. We will find the optimal flow of goods from factories, through intermediate warehouses (transshipment points), to final customer locations, with the objective of **minimizing the total transportation cost**.

<img src="licensed-image1.jpg" alt="A supply chain network diagram" height="1">

## 2. The Business Problem

A company operates a supply chain consisting of multiple factories, regional warehouses, and a wide customer base. The logistics network has the following characteristics:

- **Factories**: Each factory has a limited production/supply capacity.
- **Warehouses**: Each warehouse acts as a transshipment hub. It can receive goods from any factory and ship them to any customer. Each warehouse has a maximum handling capacity.
- **Customers**: Each customer location has a specific demand that must be met.
- **Costs**: The cost of shipping one unit of product varies for each possible route (Factory to Warehouse, and Warehouse to Customer).

The challenge is to create a detailed shipping plan that decides exactly how many units to ship along each route to satisfy all customer demands without exceeding supply or warehouse capacities, all while achieving the lowest possible total cost.

## 3. Methodology

This problem is a classic example of a **Transshipment Problem**, which is an extension of the standard transportation problem. We will formulate this as a Linear Programming (LP) model and use Python to find the optimal solution.

### Tools Used:
- **Python**: The core programming language.
- **Pandas**: For loading, manipulating, and preparing the data from CSV files.
- **Pyomo**: A powerful Python-based, open-source optimization modeling language.
- **GLPK (GNU Linear Programming Kit)**: An open-source solver that Pyomo will call to find the optimal solution to the LP problem.

## Dataset generation

In [3]:
import pandas as pd
from geopy.distance import geodesic
import random

In [53]:
#Loading Cities from CSV
cities_df = pd.read_csv('Indian Cities Database.csv')
cities_df.columns = cities_df.columns.str.strip()
cities_df['City'] = cities_df['City'].str.strip()
cities = {row['City']: (row['Lat'], row['Long']) for index, row in cities_df.iterrows()}

print(cities_df.head(10))

         City        Lat       Long country iso2          State
0      Abohar  30.144533  74.195520   India   IN         Punjab
1    Adilabad  19.400000  78.310000   India   IN      Telangana
2    Agartala  23.836049  91.279386   India   IN        Tripura
3        Agra  27.187935  78.003944   India   IN  Uttar Pradesh
4  Ahmadnagar  19.094571  74.738432   India   IN    Maharashtra
5   Ahmedabad  23.025793  72.587265   India   IN        Gujarat
6      Aizawl  23.736701  92.714596   India   IN        Mizoram
7       Ajmer  26.452103  74.638667   India   IN      Rajasthan
8       Akola  20.709569  76.998103   India   IN    Maharashtra
9   Alappuzha   9.494647  76.331108   India   IN         Kerala


In [25]:
#creating customer demand dataset

#targeted cities
customer_cities = ['Mumbai', 'Kolkata', 'Pune', 'Jaipur', 'Bengaluru', 'Hyderabad', 'Gangtok', 'Shimla', 'Lucknow', 'Bhopal']

list_of_orders = pd.read_csv('List of Orders.csv')
order_details = pd.read_csv('Order Details.csv')
    
merged_data = pd.merge(order_details, list_of_orders, on='Order ID', how='left')

demand = merged_data.groupby('City')['Quantity'].sum().reset_index()
demand_dict = demand.set_index('City')['Quantity'].to_dict()

city_demand = pd.DataFrame({
    'City_ID' : [f'C{i+1}' for i in range(len(customer_cities))],
    'City': customer_cities,
    'Latitude': [cities[c][0] for c in customer_cities],
    'Longitude': [cities[c][1] for c in customer_cities],
    'Demand': [demand_dict.get(city, 0) for city in customer_cities]
})

print(city_demand)

  City_ID       City   Latitude  Longitude  Demand
0      C1     Mumbai  18.987807  72.836447     727
1      C2    Kolkata  22.562627  88.363044     216
2      C3       Pune  18.513271  73.849852     329
3      C4     Jaipur  26.913312  75.787872     167
4      C5  Bengaluru  12.977063  77.587106     180
5      C6  Hyderabad  17.384052  78.456355     146
6      C7    Gangtok  27.325739  88.612155      93
7      C8     Shimla  31.104423  77.166623     113
8      C9    Lucknow  26.839281  80.923133     150
9     C10     Bhopal  23.254688  77.402892     247


In [29]:
#total deamnd
total_demand = city_demand['Demand'].sum()
total_demand

2368

In [33]:
#creating factories dataset
factories = ['Delhi', 'Chennai', 'Surat']

factory_data = pd.DataFrame({
    'Factory_ID': [f'F{i+1}' for i in range(len(factories))],
    'City': factories,
    'Latitude': [cities[c][0] for c in factories],
    'Longitude': [cities[c][1] for c in factories],
    'Supply_Capacity': [random.randint(500, 1500) for _ in factories]
})

print(factory_data)

  Factory_ID     City   Latitude  Longitude  Supply_Capacity
0         F1    Delhi  28.651952  77.231495             1270
1         F2  Chennai  13.084622  80.248357              516
2         F3    Surat  21.195944  72.830232             1185


In [35]:
#total supply
total_supply = factory_data['Supply_Capacity'].sum()
total_supply

2971

In [41]:
#creating warehouse dataset
warehouses = ['Chandigarh', 'Indore']

warehouse_data = pd.DataFrame({
    'Warehouse_ID': [f'W{i+1}' for i in range(len(warehouses))],
    'City': warehouses,
    'Latitude': [cities[c][0] for c in warehouses],
    'Longitude': [cities[c][1] for c in warehouses],
    'Handling_Capacity': [random.randint(800, 2000) for _ in warehouses]
})

print(warehouse_data)

  Warehouse_ID        City   Latitude  Longitude  Handling_Capacity
0           W1  Chandigarh  30.736292  76.788398               1718
1           W2      Indore  22.717736  75.858590               1740


In [43]:
#Computing Transportation Cost Matrix 
routes = []
cost_per_km = 3  

# Factory -> Warehouse routes
for f_city in factories:
    for w_city in warehouses:
        distance = geodesic(cities[f_city], cities[w_city]).km
        cost = round(distance * cost_per_km, 2)
        routes.append(('Factory', f_city, 'Warehouse', w_city, cost))

# Warehouse -> Customer routes
for w_city in warehouses:
    for c_city in customer_cities:
        distance = geodesic(cities[w_city], cities[c_city]).km
        cost = round(distance * cost_per_km, 2)
        routes.append(('Warehouse', w_city, 'Customer', c_city, cost))
    
transport_costs = pd.DataFrame(routes, columns=['From_Type', 'From_Node', 'To_Type', 'To_Node', 'Cost_per_Unit'])
print(transport_costs)

    From_Type   From_Node    To_Type     To_Node  Cost_per_Unit
0     Factory       Delhi  Warehouse  Chandigarh         704.97
1     Factory       Delhi  Warehouse      Indore        2015.05
2     Factory     Chennai  Warehouse  Chandigarh        5960.13
3     Factory     Chennai  Warehouse      Indore        3488.90
4     Factory       Surat  Warehouse  Chandigarh        3385.72
5     Factory       Surat  Warehouse      Indore        1065.88
6   Warehouse  Chandigarh   Customer      Mumbai        4082.95
7   Warehouse  Chandigarh   Customer     Kolkata        4391.99
8   Warehouse  Chandigarh   Customer        Pune        4158.16
9   Warehouse  Chandigarh   Customer      Jaipur        1304.44
10  Warehouse  Chandigarh   Customer   Bengaluru        5904.83
11  Warehouse  Chandigarh   Customer   Hyderabad        4465.66
12  Warehouse  Chandigarh   Customer     Gangtok        3634.28
13  Warehouse  Chandigarh   Customer      Shimla         163.57
14  Warehouse  Chandigarh   Customer    

In [49]:
#Export and display
factory_data.to_csv('factories.csv', index=False)
warehouse_data.to_csv('warehouses.csv', index=False)
city_demand.to_csv('customers.csv', index=False)
transport_costs.to_csv('transport_costs.csv', index=False)

print("Dataset generation complete!\n")
print("--- Factories ---")
print(factory_data, "\n")
print("--- Warehouses ---")
print(warehouse_data, "\n")
print("--- City Demands ---")
print(city_demand, "\n")
print("---Transport Costs ---")
print(transport_costs.head())

Dataset generation complete!

--- Factories ---
  Factory_ID     City   Latitude  Longitude  Supply_Capacity
0         F1    Delhi  28.651952  77.231495             1270
1         F2  Chennai  13.084622  80.248357              516
2         F3    Surat  21.195944  72.830232             1185 

--- Warehouses ---
  Warehouse_ID        City   Latitude  Longitude  Handling_Capacity
0           W1  Chandigarh  30.736292  76.788398               1718
1           W2      Indore  22.717736  75.858590               1740 

--- City Demands ---
  City_ID       City   Latitude  Longitude  Demand
0      C1     Mumbai  18.987807  72.836447     727
1      C2    Kolkata  22.562627  88.363044     216
2      C3       Pune  18.513271  73.849852     329
3      C4     Jaipur  26.913312  75.787872     167
4      C5  Bengaluru  12.977063  77.587106     180
5      C6  Hyderabad  17.384052  78.456355     146
6      C7    Gangtok  27.325739  88.612155      93
7      C8     Shimla  31.104423  77.166623     113
8

## 4. Data Preparation

In this step, we import the necessary libraries and load our datasets from CSV files. The data is structured into four main components:
1.  **Factories**: Locations and supply capacities.
2.  **Warehouses**: Locations and handling capacities.
3.  **Customers**: Locations and demand requirements.
4.  **Transport Costs**: The cost per unit for each valid shipping lane.

In [92]:
import pandas as pd
import pyomo.environ as pyo

#Loading Data from CSV files
factories_df = pd.read_csv('factories.csv')
warehouses_df = pd.read_csv('warehouses.csv')
customers_df = pd.read_csv('customers.csv')
costs_df = pd.read_csv('transport_costs.csv')

#Data Preparation
# Creating dictionaries for easy lookup of capacities, demands, and costs.
supply = factories_df.set_index('City')['Supply_Capacity'].to_dict()
demand = customers_df.set_index('City')['Demand'].to_dict()
warehouse_capacity = warehouses_df.set_index('City')['Handling_Capacity'].to_dict()

# Extracting sets for the model
factories = list(factories_df['City'])
warehouses = list(warehouses_df['City'])
customers = list(customers_df['City'])

# Creating a nested dictionary for transport costs: costs[from][to]
costs = {}
for _, row in costs_df.iterrows():
    from_node = row['From_Node']
    to_node = row['To_Node']
    cost_val = row['Cost_per_Unit']
    if from_node not in costs:
        costs[from_node] = {}
    costs[from_node][to_node] = cost_val

## 5. Model Formulation with Pyomo

Now, we define the mathematical model using Pyomo's syntax. This involves specifying the sets, decision variables, objective function, and constraints.

In [94]:
#Pyomo Model Formulation
model = pyo.ConcreteModel()

#Defining Sets
#These are the indices for our variables and parameters.
model.factories = pyo.Set(initialize=factories)
model.warehouses = pyo.Set(initialize=warehouses)
model.customers = pyo.Set(initialize=customers)

#Defining Decision Variables
# x[f, w]: Quantity shipped from factory f to warehouse w
model.x = pyo.Var(model.factories, model.warehouses, within=pyo.NonNegativeReals)
# y[w, c]: Quantity shipped from warehouse w to customer c
model.y = pyo.Var(model.warehouses, model.customers, within=pyo.NonNegativeReals)

#Defining Objective Function
#The goal is to minimize the total transportation cost.
def total_cost_rule(model):
    cost_fw = sum(costs[f][w] * model.x[f, w] for f in model.factories for w in model.warehouses)
    cost_wc = sum(costs[w][c] * model.y[w, c] for w in model.warehouses for c in model.customers)
    return cost_fw + cost_wc
model.total_cost = pyo.Objective(rule=total_cost_rule, sense=pyo.minimize)

#Defining Constraints
# a) Supply Constraint: Do not ship more from a factory than its capacity.
def supply_rule(model, f):
    return sum(model.x[f, w] for w in model.warehouses) <= supply[f]
model.supply_constraint = pyo.Constraint(model.factories, rule=supply_rule)

# b) Demand Constraint: Ensure each customer's demand is met.
def demand_rule(model, c):
    return sum(model.y[w, c] for w in model.warehouses) >= demand[c]
model.demand_constraint = pyo.Constraint(model.customers, rule=demand_rule)

# c) Warehouse Conservation Constraint: The amount shipped out of a warehouse cannot exceed the amount shipped in.
def warehouse_flow_rule(model, w):
    in_flow = sum(model.x[f, w] for f in model.factories)
    out_flow = sum(model.y[w, c] for c in model.customers)
    return out_flow <= in_flow
model.warehouse_flow_constraint = pyo.Constraint(model.warehouses, rule=warehouse_flow_rule)

# d) Warehouse Capacity Constraint: The total flow through a warehouse cannot exceed its handling capacity.
def warehouse_capacity_rule(model, w):
    # In-flow is a good measure of what the warehouse handles.
    return sum(model.x[f, w] for f in model.factories) <= warehouse_capacity[w]
model.warehouse_capacity_constraint = pyo.Constraint(model.warehouses, rule=warehouse_capacity_rule)

## 6. Solving the Model and Analyzing Results

With the model fully defined, we can now call the GLPK solver to find the optimal solution. We will then print a clear report of the results, including:
- The minimum total cost.
- The optimal shipping plan (quantities for each route).
- The utilization rates for factories and warehouses to identify potential bottlenecks.

In [97]:
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

print("--- Transshipment Optimization Results ---")
print(f"Solver status: {results.solver.status}")
print(f"Solver termination condition: {results.solver.termination_condition}")

if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"\nOptimal Total Cost: ₹{pyo.value(model.total_cost):,.2f}\n")

    print("--- Optimal Shipping Plan ---")
    print("\nFactory to Warehouse Shipments:")
    for f in model.factories:
        for w in model.warehouses:
            if pyo.value(model.x[f, w]) > 0.01:
                print(f"  {f} -> {w}: {pyo.value(model.x[f, w]):.0f} units")

    print("\nWarehouse to Customer Shipments:")
    for w in model.warehouses:
        for c in model.customers:
            if pyo.value(model.y[w, c]) > 0.01:
                print(f"  {w} -> {c}: {pyo.value(model.y[w, c]):.0f} units")
            
    print("\n--- Node Utilization ---")
    print("\nFactory Supply Utilization:")
    for f in factories:
        used = sum(pyo.value(model.x[f,w]) for w in warehouses)
        print(f"  {f}: Used {used:.0f} / {supply[f]} units ({used/supply[f]:.1%})")

    print("\nWarehouse Capacity Utilization:")
    for w in warehouses:
        used = sum(pyo.value(model.x[f,w]) for f in factories)
        print(f"  {w}: Used {used:.0f} / {warehouse_capacity[w]} units ({used/warehouse_capacity[w]:.1%})")

else:
     print("\nCould not find an optimal solution.")

--- Transshipment Optimization Results ---
Solver status: ok
Solver termination condition: optimal

Optimal Total Cost: ₹7,103,980.28

--- Optimal Shipping Plan ---

Factory to Warehouse Shipments:
  Delhi -> Chandigarh: 739 units
  Delhi -> Indore: 444 units
  Surat -> Indore: 1185 units

Warehouse to Customer Shipments:
  Chandigarh -> Kolkata: 216 units
  Chandigarh -> Jaipur: 167 units
  Chandigarh -> Gangtok: 93 units
  Chandigarh -> Shimla: 113 units
  Chandigarh -> Lucknow: 150 units
  Indore -> Mumbai: 727 units
  Indore -> Pune: 329 units
  Indore -> Bengaluru: 180 units
  Indore -> Hyderabad: 146 units
  Indore -> Bhopal: 247 units

--- Node Utilization ---

Factory Supply Utilization:
  Delhi: Used 1183 / 1270 units (93.1%)
  Chennai: Used 0 / 516 units (0.0%)
  Surat: Used 1185 / 1185 units (100.0%)

Warehouse Capacity Utilization:
  Chandigarh: Used 739 / 1718 units (43.0%)
  Indore: Used 1629 / 1740 units (93.6%)


## 7. Conclusion & Business Insights

The optimization model successfully identified the optimal shipping strategy that minimizes total logistics costs while satisfying all supply, demand, and capacity constraints.

### Key Insights:
- **Cost Savings**: By following the optimal plan, the company can achieve a minimum transportation cost and a clear, actionable distribution strategy.
- **Bottleneck Identification**: The utilization analysis highlights which facilities are operating near their maximum capacity. For example, if a warehouse is consistently at 100% utilization, it may be a candidate for a capacity expansion. Similarly, under-utilized factories might indicate an opportunity to consolidate production.
- **Strategic Decision-Making**: This model can be adapted to test various "what-if" scenarios, such as: 
    - What happens if a new warehouse is opened?
    - How do changes in fuel prices affect the distribution plan?
    - What is the cost impact of a sudden spike in demand in a specific region?

This data-driven approach provides a powerful tool for making strategic, cost-saving decisions in supply chain management.